The purpose of this project is to take data from a craiglist car sales database and use it to predict prices for other used cars.  This first section focuses on cleaning the data and simplifying it to focus on only certain qualities of the car that can be used.

In [1]:
#Import anything that I might be using durinng this project
import pandas as pd
import seaborn as sns
import os
import pandas_profiling as profile
from sqlalchemy import create_engine
from sklearn.preprocessing import LabelEncoder

In [2]:
#Since it is large, I am already deleting several columns I know that I am not using, including URL, region, condition, fuel, title status, vin, size, type, image_url, description, county, lat, and long 
#Also I will load only the first part of the data
col_list= ['id', 'price', 'year', 'manufacturer', 'model', 'cylinders', 'odometer', 'transmission', 'drive', 'paint_color', 'state']
cars = pd.read_csv('vehicles.csv', usecols=col_list)
cars.head()


,id,price,year,manufacturer,model,cylinders,odometer,transmission,drive,paint_color,state
0,7184791621,6995,2000.0,gmc,new sierra 1500,8 cylinders,167783.0,automatic,4wd,red,mn
1,7184773187,8750,2013.0,hyundai,sonata,4 cylinders,90821.0,automatic,fwd,grey,mn
2,7193375964,10900,2013.0,toyota,prius,4 cylinders,92800.0,automatic,fwd,blue,ct
3,7195108810,12500,2003.0,mitsubishi,lancer,4 cylinders,NaN,manual,4wd,grey,nm
4,7184712241,16995,2007.0,gmc,sierra classic 2500hd,8 cylinders,254217.0,automatic,4wd,white,mn


In [3]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423857 entries, 0 to 423856
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            423857 non-null  int64  
 1   price         423857 non-null  int64  
 2   year          328743 non-null  float64
 3   manufacturer  313242 non-null  object 
 4   model         325384 non-null  object 
 5   cylinders     197679 non-null  object 
 6   odometer      270585 non-null  float64
 7   transmission  328065 non-null  object 
 8   drive         231119 non-null  object 
 9   paint_color   222203 non-null  object 
 10  state         423857 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 24.3+ MB


In [4]:
#Take a closer look at null data
missing = pd.concat([cars.isnull().sum(), 100 * cars.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count',ascending=True)

,count,%
id,0,0.000000
price,0,0.000000
state,0,0.000000
year,95114,22.440115
transmission,95792,22.600075
model,98473,23.232600
manufacturer,110615,26.097245
odometer,153272,36.161252
drive,192738,45.472412
paint_color,201654,47.575951


In [5]:
#First I am going to get rid of the cars with a price that cannot be used
cars['price'].describe()

count    4.238570e+05
mean     5.779437e+04
std      1.002581e+07
min      0.000000e+00
25%      4.499000e+03
50%      9.995000e+03
75%      1.899500e+04
max      3.808256e+09
Name: price, dtype: float64

In [6]:
cars = cars[cars['price'] !=0]
cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 388832 entries, 0 to 423856
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            388832 non-null  int64  
 1   price         388832 non-null  int64  
 2   year          300973 non-null  float64
 3   manufacturer  286718 non-null  object 
 4   model         297976 non-null  object 
 5   cylinders     184477 non-null  object 
 6   odometer      249186 non-null  float64
 7   transmission  300620 non-null  object 
 8   drive         212457 non-null  object 
 9   paint_color   205529 non-null  object 
 10  state         388832 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 25.2+ MB


In [7]:
#There are some high values listed for price, what are realistic?
cars.loc[cars.price > 200000]

,id,price,year,manufacturer,model,cylinders,odometer,transmission,drive,paint_color,state
2735,7192116434,1495000,2014.0,lexus,is 250,6 cylinders,79577.0,automatic,rwd,grey,tx
2846,7191933878,9999999,2008.0,ram,NaN,6 cylinders,NaN,automatic,rwd,white,tx
3274,7189861093,239000,2019.0,NaN,"Lamborghini Urus 2,500 MILES",8 cylinders,2500.0,automatic,4wd,NaN,ca
3727,7190798333,239999,1933.0,chrysler,NaN,NaN,NaN,automatic,NaN,NaN,ca
3778,7190627325,239999,1933.0,ford,victoria,8 cylinders,5.0,automatic,NaN,black,ca
...,...,...,...,...,...,...,...,...,...,...,...
406245,7178735030,123456789,1960.0,NaN,Renault Caravelle,NaN,NaN,manual,NaN,red,ca
406766,7192926779,999999,1998.0,NaN,IH 4900,other,NaN,manual,rwd,NaN,in
406767,7192918966,999999,2004.0,chevrolet,express,8 cylinders,NaN,manual,rwd,NaN,in
406778,7192902420,999999,1998.0,chevrolet,c6500,NaN,NaN,manual,rwd,NaN,in


In [8]:
#After looking at some of these vehicles, the lexus is not worth $18,500,000, and the values over 500000 are useless.  We will remove them
cars = cars.loc[cars.price< 500000]
cars.head()

,id,price,year,manufacturer,model,cylinders,odometer,transmission,drive,paint_color,state
0,7184791621,6995,2000.0,gmc,new sierra 1500,8 cylinders,167783.0,automatic,4wd,red,mn
1,7184773187,8750,2013.0,hyundai,sonata,4 cylinders,90821.0,automatic,fwd,grey,mn
2,7193375964,10900,2013.0,toyota,prius,4 cylinders,92800.0,automatic,fwd,blue,ct
3,7195108810,12500,2003.0,mitsubishi,lancer,4 cylinders,NaN,manual,4wd,grey,nm
4,7184712241,16995,2007.0,gmc,sierra classic 2500hd,8 cylinders,254217.0,automatic,4wd,white,mn


In [9]:
cars.loc[cars.price <100]
#These low values also seem useless, so they will be removed

,id,price,year,manufacturer,model,cylinders,odometer,transmission,drive,paint_color,state
166,7191488664,19,1986.0,NaN,Grand National,NaN,NaN,automatic,NaN,NaN,tx
243,7195024987,1,2005.0,cadillac,escalade,NaN,NaN,automatic,NaN,NaN,nm
579,7193371115,1,2010.0,gmc,sierra 1500,8 cylinders,186049.0,automatic,rwd,silver,tx
1282,7182377328,1,2012.0,NaN,All,NaN,NaN,other,NaN,NaN,fl
2400,7185992984,1,2010.0,NaN,autos,NaN,111.0,automatic,NaN,NaN,wa
...,...,...,...,...,...,...,...,...,...,...,...
422791,7193513547,5,2013.0,subaru,wrx,NaN,76000.0,manual,4wd,black,nj
422835,7185849518,1,2006.0,dodge,sprinter 2500,NaN,270000.0,automatic,NaN,NaN,va
422887,7184931843,1,1965.0,NaN,Corvair,6 cylinders,NaN,other,NaN,NaN,mi
422970,7193630852,7,2012.0,honda,civic,4 cylinders,115694.0,automatic,fwd,black,nj


In [10]:
cars = cars.loc[cars.price >100]

In [11]:
#I saw a lot of NaN values for the make, year, model, and odometer of the cars.  They would be useless for comparison since that is the core of this project, so we are going to take them out
cars = cars.dropna(subset= ['manufacturer','model','year', 'odometer'])
cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235926 entries, 0 to 423856
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            235926 non-null  int64  
 1   price         235926 non-null  int64  
 2   year          235926 non-null  float64
 3   manufacturer  235926 non-null  object 
 4   model         235926 non-null  object 
 5   cylinders     155687 non-null  object 
 6   odometer      235926 non-null  float64
 7   transmission  234891 non-null  object 
 8   drive         182725 non-null  object 
 9   paint_color   177237 non-null  object 
 10  state         235926 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 15.3+ MB


In [12]:
#The rest of the Null values are acceptable but should be replaced.  They will be given the value unknown, which can be used if needed by anyone that will be using the final model if they do not know everything about their car.
cars = cars.fillna('Unknown')
cars.head()


,id,price,year,manufacturer,model,cylinders,odometer,transmission,drive,paint_color,state
0,7184791621,6995,2000.0,gmc,new sierra 1500,8 cylinders,167783.0,automatic,4wd,red,mn
1,7184773187,8750,2013.0,hyundai,sonata,4 cylinders,90821.0,automatic,fwd,grey,mn
2,7193375964,10900,2013.0,toyota,prius,4 cylinders,92800.0,automatic,fwd,blue,ct
4,7184712241,16995,2007.0,gmc,sierra classic 2500hd,8 cylinders,254217.0,automatic,4wd,white,mn
5,7184711787,13995,2012.0,ford,f-150,6 cylinders,188406.0,automatic,4wd,grey,mn


In [13]:
#Check to make sure there are only real states and capitalize them
cars['state'].unique()
#This includes Washington DC, which I find acceptable

array(['mn', 'ct', 'nm', 'tx', 'ny', 'dc', 'nc', 'va', 'wa', 'fl', 'ga',
       'ca', 'de', 'tn', 'wi', 'or', 'oh', 'ar', 'ok', 'al', 'ia', 'nv',
       'ma', 'mo', 'pa', 'ms', 'ut', 'ky', 'la', 'ks', 'vt', 'ak', 'mt',
       'az', 'me', 'nh', 'md', 'ne', 'nd', 'co', 'id', 'wv', 'nj', 'hi',
       'mi', 'ri', 'sc', 'sd', 'wy', 'il', 'in'], dtype=object)

In [14]:
cars['state']=cars['state'].str.upper()
cars.head()

,id,price,year,manufacturer,model,cylinders,odometer,transmission,drive,paint_color,state
0,7184791621,6995,2000.0,gmc,new sierra 1500,8 cylinders,167783.0,automatic,4wd,red,MN
1,7184773187,8750,2013.0,hyundai,sonata,4 cylinders,90821.0,automatic,fwd,grey,MN
2,7193375964,10900,2013.0,toyota,prius,4 cylinders,92800.0,automatic,fwd,blue,CT
4,7184712241,16995,2007.0,gmc,sierra classic 2500hd,8 cylinders,254217.0,automatic,4wd,white,MN
5,7184711787,13995,2012.0,ford,f-150,6 cylinders,188406.0,automatic,4wd,grey,MN


In [15]:
#Give a numerical value to each object in order to use later.  Also check to make sure there are no unusual values in each.
cars['manufacturer'].unique()

array(['gmc', 'hyundai', 'toyota', 'ford', 'chevrolet', 'ram', 'buick',
       'jeep', 'subaru', 'nissan', 'dodge', 'audi', 'rover', 'lexus',
       'honda', 'mini', 'mercedes-benz', 'cadillac', 'bmw', 'kia',
       'volvo', 'jaguar', 'chrysler', 'acura', 'mitsubishi', 'mazda',
       'volkswagen', 'pontiac', 'lincoln', 'infiniti', 'saturn',
       'mercury', 'ferrari', 'fiat', 'tesla', 'land rover',
       'harley-davidson', 'datsun', 'alfa-romeo', 'morgan',
       'aston-martin', 'porche'], dtype=object)

In [16]:
encoder= LabelEncoder()
encoder.fit(cars.manufacturer)
cars["manufacturer id"] = encoder.transform(cars.manufacturer)
cars.head()

,id,price,year,manufacturer,model,cylinders,odometer,transmission,drive,paint_color,state,manufacturer id
0,7184791621,6995,2000.0,gmc,new sierra 1500,8 cylinders,167783.0,automatic,4wd,red,MN,14
1,7184773187,8750,2013.0,hyundai,sonata,4 cylinders,90821.0,automatic,fwd,grey,MN,17
2,7193375964,10900,2013.0,toyota,prius,4 cylinders,92800.0,automatic,fwd,blue,CT,39
4,7184712241,16995,2007.0,gmc,sierra classic 2500hd,8 cylinders,254217.0,automatic,4wd,white,MN,14
5,7184711787,13995,2012.0,ford,f-150,6 cylinders,188406.0,automatic,4wd,grey,MN,13


In [17]:
encoder.fit(cars.model)
cars['model id']= encoder.transform(cars.model)
cars.head()

,id,price,year,manufacturer,model,cylinders,odometer,transmission,drive,paint_color,state,manufacturer id,model id
0,7184791621,6995,2000.0,gmc,new sierra 1500,8 cylinders,167783.0,automatic,4wd,red,MN,14,11521
1,7184773187,8750,2013.0,hyundai,sonata,4 cylinders,90821.0,automatic,fwd,grey,MN,17,14842
2,7193375964,10900,2013.0,toyota,prius,4 cylinders,92800.0,automatic,fwd,blue,CT,39,12299
4,7184712241,16995,2007.0,gmc,sierra classic 2500hd,8 cylinders,254217.0,automatic,4wd,white,MN,14,14252
5,7184711787,13995,2012.0,ford,f-150,6 cylinders,188406.0,automatic,4wd,grey,MN,13,6867


In [18]:
cars['cylinders'].unique()

array(['8 cylinders', '4 cylinders', '6 cylinders', 'Unknown',
       '10 cylinders', '5 cylinders', '12 cylinders', '3 cylinders',
       'other'], dtype=object)

In [19]:
#I want to keep the cylinders number value (8 cylinders = 8).  I will give a number value for other and Unknown, and convert them to an integer.
def process_cylinders(cylinders):
    cylinder_number= cylinders.split(" ")[0]
    return cylinder_number

cars['cylinders id']=cars.cylinders.apply(process_cylinders)

In [20]:
cars['cylinders id']=cars.cylinders.apply(process_cylinders)
cars['cylinders id'] =cars['cylinders id'].replace({'other':1,'Unknown':2})
cars['cylinders id'].unique()

array(['8', '4', '6', 2, '10', '5', '12', '3', 1], dtype=object)

In [21]:
cars['cylinders id']= cars['cylinders id'].astype(str).astype(int)
cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235926 entries, 0 to 423856
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               235926 non-null  int64  
 1   price            235926 non-null  int64  
 2   year             235926 non-null  float64
 3   manufacturer     235926 non-null  object 
 4   model            235926 non-null  object 
 5   cylinders        235926 non-null  object 
 6   odometer         235926 non-null  float64
 7   transmission     235926 non-null  object 
 8   drive            235926 non-null  object 
 9   paint_color      235926 non-null  object 
 10  state            235926 non-null  object 
 11  manufacturer id  235926 non-null  int32  
 12  model id         235926 non-null  int32  
 13  cylinders id     235926 non-null  int32  
dtypes: float64(2), int32(3), int64(2), object(7)
memory usage: 18.0+ MB


In [22]:
cars['transmission'].unique()

array(['automatic', 'manual', 'other', 'Unknown'], dtype=object)

In [23]:
encoder.fit(cars.transmission)
cars['transmission id']= encoder.transform(cars.transmission)
cars.head()

,id,price,year,manufacturer,model,cylinders,odometer,transmission,drive,paint_color,state,manufacturer id,model id,cylinders id,transmission id
0,7184791621,6995,2000.0,gmc,new sierra 1500,8 cylinders,167783.0,automatic,4wd,red,MN,14,11521,8,1
1,7184773187,8750,2013.0,hyundai,sonata,4 cylinders,90821.0,automatic,fwd,grey,MN,17,14842,4,1
2,7193375964,10900,2013.0,toyota,prius,4 cylinders,92800.0,automatic,fwd,blue,CT,39,12299,4,1
4,7184712241,16995,2007.0,gmc,sierra classic 2500hd,8 cylinders,254217.0,automatic,4wd,white,MN,14,14252,8,1
5,7184711787,13995,2012.0,ford,f-150,6 cylinders,188406.0,automatic,4wd,grey,MN,13,6867,6,1


In [24]:
cars['drive'].unique()

array(['4wd', 'fwd', 'rwd', 'Unknown'], dtype=object)

In [25]:
encoder.fit(cars.drive)
cars['drive id']= encoder.transform(cars.drive)
cars.head()

,id,price,year,manufacturer,model,cylinders,odometer,transmission,drive,paint_color,state,manufacturer id,model id,cylinders id,transmission id,drive id
0,7184791621,6995,2000.0,gmc,new sierra 1500,8 cylinders,167783.0,automatic,4wd,red,MN,14,11521,8,1,0
1,7184773187,8750,2013.0,hyundai,sonata,4 cylinders,90821.0,automatic,fwd,grey,MN,17,14842,4,1,2
2,7193375964,10900,2013.0,toyota,prius,4 cylinders,92800.0,automatic,fwd,blue,CT,39,12299,4,1,2
4,7184712241,16995,2007.0,gmc,sierra classic 2500hd,8 cylinders,254217.0,automatic,4wd,white,MN,14,14252,8,1,0
5,7184711787,13995,2012.0,ford,f-150,6 cylinders,188406.0,automatic,4wd,grey,MN,13,6867,6,1,0


In [26]:
cars['paint_color'].unique()

array(['red', 'grey', 'blue', 'white', 'custom', 'silver', 'brown',
       'black', 'purple', 'Unknown', 'orange', 'green', 'yellow'],
      dtype=object)

In [27]:
encoder.fit(cars.paint_color)
cars['color id']= encoder.transform(cars.paint_color)
cars.head()

,id,price,year,manufacturer,model,cylinders,odometer,transmission,drive,paint_color,state,manufacturer id,model id,cylinders id,transmission id,drive id,color id
0,7184791621,6995,2000.0,gmc,new sierra 1500,8 cylinders,167783.0,automatic,4wd,red,MN,14,11521,8,1,0,9
1,7184773187,8750,2013.0,hyundai,sonata,4 cylinders,90821.0,automatic,fwd,grey,MN,17,14842,4,1,2,6
2,7193375964,10900,2013.0,toyota,prius,4 cylinders,92800.0,automatic,fwd,blue,CT,39,12299,4,1,2,2
4,7184712241,16995,2007.0,gmc,sierra classic 2500hd,8 cylinders,254217.0,automatic,4wd,white,MN,14,14252,8,1,0,11
5,7184711787,13995,2012.0,ford,f-150,6 cylinders,188406.0,automatic,4wd,grey,MN,13,6867,6,1,0,6


In [28]:
encoder.fit(cars.state)
cars['state id']= encoder.transform(cars.state)
cars.head()

,id,price,year,manufacturer,model,cylinders,odometer,transmission,drive,paint_color,state,manufacturer id,model id,cylinders id,transmission id,drive id,color id,state id
0,7184791621,6995,2000.0,gmc,new sierra 1500,8 cylinders,167783.0,automatic,4wd,red,MN,14,11521,8,1,0,9,23
1,7184773187,8750,2013.0,hyundai,sonata,4 cylinders,90821.0,automatic,fwd,grey,MN,17,14842,4,1,2,6,23
2,7193375964,10900,2013.0,toyota,prius,4 cylinders,92800.0,automatic,fwd,blue,CT,39,12299,4,1,2,2,6
4,7184712241,16995,2007.0,gmc,sierra classic 2500hd,8 cylinders,254217.0,automatic,4wd,white,MN,14,14252,8,1,0,11,23
5,7184711787,13995,2012.0,ford,f-150,6 cylinders,188406.0,automatic,4wd,grey,MN,13,6867,6,1,0,6,23


In [29]:
#Check for duplicated rows
cars_duplicated = cars[cars.duplicated()]
cars_duplicated

,id,price,year,manufacturer,model,cylinders,odometer,transmission,drive,paint_color,state,manufacturer id,model id,cylinders id,transmission id,drive id,color id,state id


In [30]:
#Take a final look at where we are
cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235926 entries, 0 to 423856
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               235926 non-null  int64  
 1   price            235926 non-null  int64  
 2   year             235926 non-null  float64
 3   manufacturer     235926 non-null  object 
 4   model            235926 non-null  object 
 5   cylinders        235926 non-null  object 
 6   odometer         235926 non-null  float64
 7   transmission     235926 non-null  object 
 8   drive            235926 non-null  object 
 9   paint_color      235926 non-null  object 
 10  state            235926 non-null  object 
 11  manufacturer id  235926 non-null  int32  
 12  model id         235926 non-null  int32  
 13  cylinders id     235926 non-null  int32  
 14  transmission id  235926 non-null  int32  
 15  drive id         235926 non-null  int32  
 16  color id         235926 non-null  int3

In [32]:
#Saving the data for future use
# save the data to a new csv file
cars.to_csv('cars.csv')